In [5]:
!pip install transformers

In [6]:
import pandas as pd
import numpy as np
import re

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax
import csv
import urllib.request

In [7]:
df = pd.read_csv('./data.csv')

In [13]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [ ]:
df["sentiment"] = ""
df["emotion"] = ""


scores_ls = []


for idx, row in df.iterrows():
    text = row["text"]
    if type(text) == str:
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        max_index = np.where(scores == np.amax(scores))
        df[idx, "sentiment"] = max_index[0][0] # 0 = negative, 1 = neutral, 2 = positive
        scores_ls.append(scores)
        

In [ ]:
print(scores_ls)

In [ ]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [ ]:
df.to_csv("afterfirstscores.csv")

In [ ]:
# other emotions woooo

task2='emotion'
MODEL2 = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer2 = AutoTokenizer.from_pretrained(MODEL)

labels2=[]
mapping_link2 = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link2) as f2:
    html2 = f2.read().decode('utf-8').split("\n")
    csvreader2 = csv.reader(html2, delimiter='\t')
labels2 = [row[1] for row in csvreader2 if len(row) > 1]

model2 = AutoModelForSequenceClassification.from_pretrained(MODEL2)
model2.save_pretrained(MODEL2)

In [44]:
scores_ls_2 = []


for idx, row in df.iterrows():
    text = row["text"]
    if type(text) == str:
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores_2 = output[0][0].detach().numpy()
        scores_2 = softmax(scores_2)
        scores_ls_2.append(scores_2)
        max_index = np.where(scores == np.amax(scores))
        df[idx, "emotion"] = max_index[0][0]

In [4]:
df.to_csv("aftersecondscores.csv")

NameError: name 'df' is not defined

In [45]:
print(scores_ls_2)

[array([0.07467674, 0.32975185, 0.59557146], dtype=float32), array([0.9154226 , 0.07712659, 0.00745069], dtype=float32), array([0.34333664, 0.6055438 , 0.05111959], dtype=float32), array([0.02158527, 0.8676438 , 0.11077087], dtype=float32), array([0.00713694, 0.03008205, 0.96278095], dtype=float32), array([0.01675407, 0.76507246, 0.21817347], dtype=float32), array([0.3143857 , 0.6382404 , 0.04737387], dtype=float32), array([0.0012352, 0.0096351, 0.9891294], dtype=float32), array([0.00407512, 0.50897837, 0.4869465 ], dtype=float32), array([0.00893207, 0.10496762, 0.88610023], dtype=float32), array([0.03356065, 0.86635065, 0.10008869], dtype=float32), array([0.05675641, 0.69115263, 0.2520909 ], dtype=float32), array([0.00125905, 0.02646568, 0.9722753 ], dtype=float32), array([0.06226735, 0.8503528 , 0.08737992], dtype=float32), array([0.0214071 , 0.60179794, 0.3767949 ], dtype=float32), array([0.04770446, 0.885613  , 0.06668252], dtype=float32), array([0.00511449, 0.1472824 , 0.8476031 ]

In [46]:
def find_sent_emot(dta):
    
    num_neg = 0
    num_pos = 0
    num_neutral = 0

    num_anger = 0
    num_joy = 0
    num_optimism = 0
    num_sad = 0

    for idx, row in df.iterrows():
        sentiment = row["sentiment"]
        emotion = row["emotion"]
        if sentiment == 0: 
            num_neg += 1

        if sentiment == 1:
            num_neutral += 1

        if sentiment == 2: 
            num_pos += 1

        if emotion == 0: 
            num_anger += 1

        if emotion == 1: 
            num_joy += 1

        if emotion == 2: 
            num_optimism += 1 

        if emotion == 3:
            num_sad += 1
            
        neg_percent = num_neg/df.size
        pos_percent = num_pos/df.size
        neut_percent = num_neutral/df.size

        anger_percent = num_anger/df.size
        joy_percent = num_joy/df.size
        optimism_percent = num_optimism/df.size
        sad_percent = num_sad/df.size
        
    return [num_neg, num_pos, num_neutral, num_anger, num_joy, num_optimism, num_sad, neg_percent, pos_percent, neut_percent, anger_percent, joy_percent, optimism_percent, sad_percent]

In [47]:
# just find the overall sentiment and emotion for all time

general_nums = find_sent_emot(df)

In [48]:
print(scores_ls_2)

[array([0.07467674, 0.32975185, 0.59557146], dtype=float32), array([0.9154226 , 0.07712659, 0.00745069], dtype=float32), array([0.34333664, 0.6055438 , 0.05111959], dtype=float32), array([0.02158527, 0.8676438 , 0.11077087], dtype=float32), array([0.00713694, 0.03008205, 0.96278095], dtype=float32), array([0.01675407, 0.76507246, 0.21817347], dtype=float32), array([0.3143857 , 0.6382404 , 0.04737387], dtype=float32), array([0.0012352, 0.0096351, 0.9891294], dtype=float32), array([0.00407512, 0.50897837, 0.4869465 ], dtype=float32), array([0.00893207, 0.10496762, 0.88610023], dtype=float32), array([0.03356065, 0.86635065, 0.10008869], dtype=float32), array([0.05675641, 0.69115263, 0.2520909 ], dtype=float32), array([0.00125905, 0.02646568, 0.9722753 ], dtype=float32), array([0.06226735, 0.8503528 , 0.08737992], dtype=float32), array([0.0214071 , 0.60179794, 0.3767949 ], dtype=float32), array([0.04770446, 0.885613  , 0.06668252], dtype=float32), array([0.00511449, 0.1472824 , 0.8476031 ]

In [49]:
df_pfizer = df[df['pfizer'] == 1]
df_moderna = df[df['moderna'] == 1]

In [50]:
pfizer_nums = find_sent_emot(df_pfizer)
modern_nums = find_sent_emot(df_moderna)

In [58]:
# create month column

df["month"] = ""
for idx, row in df.iterrows():
    date = row["date"]
    date = str(date)
    split_hyp = date.split("-")
    if (len(split_hyp) > 2):
        month = split_hyp[1]
        df[idx, "month"] = month


In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

# group by month 

month_grouped = df.groupby('month')['sentiment'].count().reset_index()
month_grouped['month_percentage'] = 100 * month_grouped['sentiment']/month_grouped['sentiment'].sum()

month_grouped_neg = month_grouped[month_grouped['sentiment'] == 0]
month_grouped_pos = month_grouped[month_grouped['sentiment'] == 2]
month_grouped_neut = month_grouped[month_grouped['sentiment'] == 1]

plt.plot(month_grouped_neg["month"], month_grouped_neg["month_percentage"], label = "Negative")
plt.plot(month_grouped_neut["month"], month_grouped_neut["month_percentage"], label = "Neutral")
plt.plot(month_grouped_pos["month"], month_grouped_pos["month_percentage"], label = "Positive")

plt.legend()
plt.show()

NameError: name 'df' is not defined

In [ ]:
# group by month 

month_grouped = df.groupby('month')['emotion'].count().reset_index()
month_grouped['month_percentage'] = 100 * month_grouped['emotion']/month_grouped['emotion'].sum()

month_grouped_anger = month_grouped[month_grouped['emotion'] == 0]
month_grouped_joy = month_grouped[month_grouped['emotion'] == 1]
month_grouped_opt = month_grouped[month_grouped['emotion'] == 2]
month_grouped_sadness = month_grouped[month_grouped['emotion'] == 3]


plt.plot(month_grouped_anger["month"], month_grouped_anger["month_percentage"], label = "Anger")
plt.plot(month_grouped_joy["month"], month_grouped_joy["month_percentage"], label = "Joy")
plt.plot(month_grouped_opt["month"], month_grouped_opt["month_percentage"], label = "Optimism")
plt.plot(month_grouped_sadness["month"], month_grouped_sadness["month_percentage"], label = "Sadness")

plt.legend()
plt.show()

In [ ]:
# positive comparison 

df_pfizer["month"] = ""
for idx, row in df_pfizer.iterrows():
    date = row["date"]
    split_hyp = date.split("-")
    month = split_hyp[1]
    df_pfizer[idx, "month"] = month
    
df_moderna["month"] = ""
for idx, row in df_moderna.iterrows():
    date = row["date"]
    split_hyp = date.split("-")
    month = split_hyp[1]
    df_moderna[idx, "month"] = month

month_grouped_pfizer = df_pfizer.groupby('month')['sentiment'].count().reset_index()
month_grouped_pfizer['month_percentage'] = 100 * month_grouped_pfizer['sentiment']/month_grouped_pfizer['sentiment'].sum()

month_grouped_pos_pfizer = month_grouped_pfizer[month_grouped_pfizer['sentiment'] == 2]

month_grouped_moderna = df_moderna.groupby('month')['sentiment'].count().reset_index()
month_grouped_moderna['month_percentage'] = 100 * month_grouped_moderna['sentiment']/month_grouped_moderna['sentiment'].sum()

month_grouped_pos_moderna = month_grouped_moderna[month_grouped_moderna['sentiment'] == 2]

plt.plot(month_grouped_pos_moderna["month"], month_grouped_pos_moderna["month_percentage"], label = "Moderna")
plt.plot(month_grouped_pos_pfizer["month"], month_grouped_pos_pfizer["month_percentage"], label = "Pfizer")

plt.legend()
plt.show()